In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

In [73]:
def CalcRevenue(data):
    solutions  = list()
    
    prices = data.iloc[:,:4]
    production = data.iloc[:,4:]
    
    for i,row in prices.iterrows():
        
        hourlyRevMod = {}
        for j in range(production.shape[1]-1):
            
            hourlyRev = row['DA_PriceEUR'] * production.iloc[i,j+1]
            
            if production.iloc[i,j+1] > production.iloc[i,0]:
                hourlyRev = hourlyRev - row['BalancingPriceUpEUR'] * (production.iloc[i,j+1]-production.iloc[i,0])
            elif production.iloc[i,0] > production.iloc[i,j+1]:
                hourlyRev = hourlyRev + row['BalancingPriceDownEUR'] * (production.iloc[i,0]-production.iloc[i,j+1])
              
            hourlyRevMod[production.columns[j + 1]] = hourlyRev
           
        hourlyRevMod['HourDK'] = row['HourDK']
        solutions.append(hourlyRevMod)    
    
    solutions = pd.DataFrame(solutions)
    return solutions

In [74]:
def VisProduction(production):
    num_columns_to_plot = production.shape[1] - 1  # Exclude 'actProd' from the count
    fig, axes = plt.subplots(num_columns_to_plot, 1, figsize=(10, 5 * num_columns_to_plot))

    # Loop through each column after 'actProd'
    for j in range(1, production.shape[1]):  # Start from index 1 to skip 'actProd'
        label = str(production.columns[j])  # Get the column label
        color = (np.random.rand(), np.random.rand(), np.random.rand())
        axes[j-1].plot(production.index, production.iloc[:, j], label=label, marker='o', color=color)  # Plot forecast
        axes[j-1].plot(production.index, production['actProd'], label="Actual Production", marker='o', color='orange')  # Plot actual production
        axes[j-1].set_title(f'{label} vs Actual Production')  # Set title for the subplot
        axes[j-1].set_xlabel('Time')  # Set x-axis label
        axes[j-1].set_ylabel('Power Production')  # Set y-axis label
        axes[j-1].grid()  # Add a grid
        axes[j-1].legend()
    
    plt.title('Production vs. Prediction')
    plt.xlabel('Time')
    plt.ylabel('Energy')
    plt.legend()  # Show the legend
    plt.grid() 
    # Adjust layout to prevent overlap
    plt.tight_layout()
    plt.show()

In [75]:
def VisPrices(prices):
    
    fig = plt.figure(figsize=(10, 5))
    for j in range(0, prices.shape[1]):
        label = str(prices.columns[j])  # Get the column label
        color = (np.random.rand(), np.random.rand(), np.random.rand())
        plt.plot(prices.index, prices.iloc[:, j], label=label, marker='o', color=color)
    
    plt.title('Price development')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()  
    plt.grid() 
    plt.show()

In [76]:
def VisRevenue(solutions):
    
    fig, ax = plt.subplots(figsize=(12, 6))

    # Die Positionen für die Balken festlegen
    bar_width = 0.25  # Breite der Balken
    x = np.arange(len(solutions))  # x-Positionen der Balken

    # Balken für jede Datenreihe zeichnen
    for i, column in enumerate(solutions.columns):
        ax.bar(x + (i - 1) * bar_width, solutions[column], width=bar_width, label=column)

    # Achsen und Titel einstellen
    ax.set_xlabel('Time')
    ax.set_ylabel('Revenue')
    ax.set_title('Revenues based on different ML Models')
    ax.set_xticks(x)
    ax.set_xticklabels(solutions.index.strftime('%Y-%m-%d %H:%M:%S'), rotation=45, ha='right')  # Zeitstempel formatieren
    ax.legend()  # Legende hinzufügen

    # Layout anpassen und Diagramm anzeigen
    plt.tight_layout()
    plt.show()

In [77]:
def Visualise(solutions,data):
    
    prices = data.iloc[:,:4]
    production = data.iloc[:,4:]
    production['HourDK'] = prices['HourDK']
    
    prices.set_index('HourDK', inplace=True)
    production.set_index('HourDK', inplace=True)
    solutions.set_index('HourDK', inplace=True)
    
    VisProduction(production)
    VisPrices(prices)
    
    VisRevenue(solutions)

In [78]:
# p_t_hat as decision variable
def OptimizationProblemEnergybid(prices_df):
    model = gp.Model()
    solutions  = list()
    capacity = 30000.0
    
    for index, row in prices_df.iterrows():
        
        p_t_hat = model.addVar(name="p_t_hat", lb=0, ub=capacity, vtype=GRB.INTEGER)
        z_down = model.addVar(name="z_down", vtype=GRB.CONTINUOUS, lb=0)
        z_up = model.addVar(name="z_up", vtype=GRB.CONTINUOUS, lb=0)
        
        condition_down = model.addVar(name="condition_down", vtype=GRB.BINARY)
        condition_up = model.addVar(name="condition_up", vtype=GRB.BINARY)
        
        model.addConstr(z_up <= (p_t_hat - row['p_t']) * condition_up)
        model.addConstr(z_up >= (p_t_hat - row['p_t']) * condition_up)
        model.addConstr(z_down <= (row['p_t'] - p_t_hat) * condition_down)
        model.addConstr(z_down >= (row['p_t'] - p_t_hat) * condition_down)
        model.addConstr(condition_up + condition_down == 1)
        
    
        #defining the constraints
        model.addConstr(-p_t_hat <= 0)
        model.addConstr(p_t_hat <= capacity)
        model.addConstr(-z_down <= 0)
        model.addConstr(-z_up <= 0)
        
        model.setObjective((row['DA_PriceEUR']*p_t_hat)+(row['BalancingPriceDownEUR']*z_down-row['BalancingPriceUpEUR']*z_up), GRB.MAXIMIZE)
        
        model.optimize()
    
        if model.status == GRB.OPTIMAL:
            print("Optimal solution found!")
            print(f"Objective Value: {model.ObjVal}")
            values = []
            values.append({ "Day-Ahead-Price": row['DA_PriceEUR'],
                            "BalancingPriceUp":row['BalancingPriceUpEUR'],
                            "BalancingPriceDown":row['BalancingPriceDownEUR'],
                            "p_t":row['p_t'],
                            "Revenue":model.ObjVal})
            for v in model.getVars():
                print(f"{v.varName}: {v.x}")
                values.append({v.varName:v.x})
        
            
            solutions.append({row['HourDK']:values})
                
        elif model.status == GRB.INFEASIBLE:
            print("Model is infeasible.")
            model.computeIIS()
            model.write("infeasible.ilp")  # Write IIS to a file for review
            for c in model.getConstrs():
                if c.IISConstr:
                    print(f"Infeasible constraint: {c.constrName}")
            
            
        elif model.status == GRB.UNBOUNDED:
            print("Model is unbounded.")
        else:
            print("Model status:", model.status)
            
        model.remove(model.getConstrs())
        model.remove(model.getVars())
        
    return solutions

In [79]:
#Helping Method to limit the  
def ObservationPeriod(Start_Observation, End_Observation, data):
    Start_Observation = pd.to_datetime(Start_Observation)
    End_Observation = pd.to_datetime(End_Observation)
    condition = (data["HourDK"]>= Start_Observation) & (data["HourDK"]< End_Observation)
    data  = data[condition]
    return data
        

In [80]:
prices_df = pd.read_csv('./Data assignment 1/prices_merged_df_output.csv', delimiter=',')

features_df = pd.read_csv('./Data assignment 1/Feature data.csv', delimiter=',')
features_df['AKI Kalby Active Power'] = -1 * features_df['AKI Kalby Active Power']
predicted_data = pd.read_csv("./Data assignment 1/power_prod_ReguReg.csv", delimiter=',')
data = prices_df.copy()
data['actProd'] = features_df['AKI Kalby Active Power']
data["ReguReg"] = -1 * predicted_data['ReguReg']
data["HourDK"] = pd.to_datetime(data["HourDK"])
#Hier irgendwo muss ich die eigentlichen Zeitreihen hinklatschen

data = ObservationPeriod("2022-01-05 00:00:00", "2022-01-06 00:00:00", data)

#Here is the interface to get the prediction timeseries
p_t1, p_t2, p_t3 = (np.random.uniform(0.0, 6000, size=(24, 1)) for _ in range(3))
p_t1, p_t2, p_t3 = (np.round(arr, 2) for arr in (p_t1, p_t2, p_t3))

#data['LinReg'] = p_t1
#data['NonLinReg'] = p_t2

print(data)
data.reset_index(drop=True, inplace=True)
#print(data)
#solutions = CalcRevenue(data)


                 HourDK  BalancingPriceUpEUR  BalancingPriceDownEUR  \
96  2022-01-05 00:00:00             0.093867               0.093867   
97  2022-01-05 01:00:00             0.073989               0.051689   
98  2022-01-05 02:00:00             0.050039               0.050039   
99  2022-01-05 03:00:00             0.049528               0.049528   
100 2022-01-05 04:00:00             0.050069               0.050069   
101 2022-01-05 05:00:00             0.093387               0.093387   
102 2022-01-05 06:00:00             0.099157               0.099157   
103 2022-01-05 07:00:00             0.153495               0.101147   
104 2022-01-05 08:00:00             0.159995               0.129556   
105 2022-01-05 09:00:00             0.148996               0.121087   
106 2022-01-05 10:00:00             0.148996               0.135047   
107 2022-01-05 11:00:00             0.148996               0.129986   
108 2022-01-05 12:00:00             0.148996               0.131967   
109 20

In [81]:
# Hourly Optimmization Problem
data['p_t'] = p_t1
solutions = OptimizationProblemEnergybid(data)
solutions

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900HX, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Academic license 2557985 - for non-commercial use only - registered to s2___@dtu.dk
Optimize a model with 5 rows, 5 columns and 6 nonzeros
Model fingerprint: 0x88f730a2
Model has 4 quadratic constraints
Variable types: 2 continuous, 3 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 4e+03]
  Objective range  [9e-02, 9e-02]
  Bounds range     [1e+00, 3e+04]
  RHS range        [1e+00, 3e+04]
Presolve removed 4 rows and 1 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 11 nonzeros
Presolved model has 2 SOS constraint(s)
Variable types: 2 continuous, 5 integer (2 binary)
Found heuristic solution: objective 355.2181600
Found heuristic solution: objecti

[{Timestamp('2022-01-05 00:00:00'): [{'Day-Ahead-Price': 0.09387,
    'BalancingPriceUp': 0.093866959,
    'BalancingPriceDown': 0.093866959,
    'p_t': 3784.15,
    'Revenue': 355.29788289985026},
   {'p_t_hat': 30000.0},
   {'z_down': 0.0},
   {'z_up': 26215.85},
   {'condition_down': 0.0},
   {'condition_up': 1.0}]},
 {Timestamp('2022-01-05 01:00:00'): [{'Day-Ahead-Price': 0.05169,
    'BalancingPriceUp': 0.073989029,
    'BalancingPriceDown': 0.0516885339999999,
    'p_t': 2553.14,
    'Revenue': 131.97180639475886},
   {'p_t_hat': 2553.0},
   {'z_down': 0.13999999997759005},
   {'z_up': 0.0},
   {'condition_down': 1.0},
   {'condition_up': -0.0}]},
 {Timestamp('2022-01-05 02:00:00'): [{'Day-Ahead-Price': 0.05004,
    'BalancingPriceUp': 0.050038986,
    'BalancingPriceDown': 0.050038986,
    'p_t': 1861.9,
    'Revenue': 93.19800803340013},
   {'p_t_hat': 30000.0},
   {'z_down': 0.0},
   {'z_up': 28138.1},
   {'condition_down': 0.0},
   {'condition_up': 1.0}]},
 {Timestamp('2022-0

In [85]:
solutions = OptimizationProblemEnergybid(data)

revenues = []
for solution in solutions:
    for timestamp, values in solution.items():
        for value in values:
            if "Revenue" in value:
                revenues.append(value["Revenue"])

total_revenue = sum(revenues)

print(f"Total Revenue for the year: {total_revenue}")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900HX, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Academic license 2557985 - for non-commercial use only - registered to s2___@dtu.dk
Optimize a model with 5 rows, 5 columns and 6 nonzeros
Model fingerprint: 0x0a94e1da
Model has 4 quadratic constraints
Variable types: 2 continuous, 3 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 5e+03]
  Objective range  [9e-02, 9e-02]
  Bounds range     [1e+00, 3e+04]
  RHS range        [1e+00, 3e+04]
Presolve removed 4 rows and 1 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 11 nonzeros
Presolved model has 2 SOS constraint(s)
Variable types: 2 continuous, 5 integer (2 binary)
Found heuristic solution: objective 448.1985092
Found heuristic solution: objecti